In [1]:
import glm_utils.preprocessing, glm_utils.bases
import sklearn.model_selection, sklearn.metrics, sklearn.linear_model
import numpy as np
from scipy.stats import binned_statistic, pearsonr
import os
import pickle
import pandas as pd
from DN_tools import load_into_pandas, bases_dict, load_recording, get_xy, chunked_test_train_split
from tqdm import tqdm

# Notebook will run all GLM models for the requested neuron list, saving each model in the specified folder "datefolder" with their unique experiment name and name of variable predicted in the model.

# Parameters

In [ ]:
# general run parameters (TO BE CHANGED)
raw_data_dir_path = 'W:/apalaci/code/janache' # TO BE CHANGED TO LOCATION OF RAW DATA
datefolder = '../res/models' # TO BE CHANGED TO LOCATION OF RESULTS DATA
neuron_list =['DN_SPEEDO','MDN','real_DNp17','imposter'] # TO BE CHANGED TO NAME OF NEURONS ACCORDING TO DATA FOLDERS AND INFO TABLES
redo_analysis = False  # MAKE TRUE FOR RUNNING THE ANALYSIS


# general run parameters
y_names = ['v_fwd','v_ang','abs_v_fwd','abs_v_ang','pos_v_fwd','pos_v_ang','neg_v_fwd','neg_v_ang']

# downsampling parameters
sample_frequency = 20000
bin_width = 100
decimating_values = [10,10] # product must equal bin_width !

# chunk parameters
block_size = 5_000
n_block_min=5

# test-train split 
test_size = 0.35

# random seed
random_state = 42

# starting number of bins for nonlinear estimation
starting_nbins = 32

# GLM window size (number of downsampled bins: for bin_width 100 and sample_frequency 20000, downsampled bin = 5ms, thus window 600 = 3s)
window = 600

# Load dataframe

In [3]:
df = load_into_pandas(dir_path=raw_data_dir_path, neuron_list=neuron_list)
df = df[df.to_ignore == False].reset_index(drop=True)
df

,filename,#Fly,#Trial,#Cell,side,DN,to_ignore,abs_file_path
0,2022_06_08_0004,1,1,1,right,imposter,False,W:/apalaci/code/janache/DN_SPEEDO/2022_06_08_0...
1,2022_06_08_0005,1,2,1,right,imposter,False,W:/apalaci/code/janache/DN_SPEEDO/2022_06_08_0...
2,2022_07_04_0001,5,1,1,undefined,imposter,False,W:/apalaci/code/janache/DN_SPEEDO/2022_07_04_0...
3,2022_07_04_0014,5,1,2,left,imposter,False,W:/apalaci/code/janache/DN_SPEEDO/2022_07_04_0...
4,2022_07_04_0021,5,2,2,undefined,imposter,False,W:/apalaci/code/janache/DN_SPEEDO/2022_07_04_0...
...,...,...,...,...,...,...,...,...
65,2024_11_28_0003,18,1,1,right,imposter,False,W:/apalaci/code/janache/imposter/2024_11_28_00...
66,2024_11_29_0000,19,1,1,right,imposter,False,W:/apalaci/code/janache/imposter/2024_11_29_00...
67,2024_11_29_0001,19,2,1,right,imposter,False,W:/apalaci/code/janache/imposter/2024_11_29_00...
68,2024_11_29_0005,20,1,1,left,imposter,False,W:/apalaci/code/janache/imposter/2024_11_29_00...


# Analysis

In [ ]:
if redo_analysis:
    B = glm_utils.bases.raised_cosine(neye = bases_dict['neye'], ncos = bases_dict['ncos'], kpeaks = bases_dict['kpeaks'], b = bases_dict['b'], nbasis = bases_dict['nbasis'])
    B = B[-window:]
    basis_projection = glm_utils.preprocessing.BasisProjection(B)

    for index, row in tqdm(df.iterrows(), total=len(df)):
        filename = row['filename']
        csv_path = row['abs_file_path']

        singleDN_df = load_recording(csv_path=csv_path)

        x, ys = get_xy(singleDN_df,y_names)

        y_means = np.mean(ys, axis=0)
        y_stds = np.std(ys, axis=0)

        X, y_m = glm_utils.preprocessing.time_delay_embedding(x, ys, window_size=window, flatten_inside_window=True, exclude_t0=True)
        X_b = basis_projection.transform(X)
        
        X_train, X_test, y_train, y_test = chunked_test_train_split(X_b,y_m,block_size=block_size,n_block_min=n_block_min,test_size=test_size,random_state=random_state)

        for iv, varname in enumerate(y_names):
            # Fit linear model
            lr = sklearn.linear_model.LassoCV(max_iter=20000)
            lr.fit(X_train, y_train[:,iv])

            # Get linear predictions
            y_pred = lr.predict(X_train)
            y_pred_test = lr.predict(X_test)

            # Estimate nonlinearity
            nbins = starting_nbins
            not_finished = True
            while (nbins != 5) and not_finished:
                bin_edges_quantilebased = np.quantile(y_pred, np.linspace(0, 1, nbins + 1))
                try:
                    statistic, bin_edges, binnumber = binned_statistic(y_pred, y_train[:,iv], statistic='mean', bins=bin_edges_quantilebased, range=None)
                    bin_centers = bin_edges[:-1] + np.median(np.diff(bin_edges)) / 2
                    constant_input = False
                    not_finished = False
                except ValueError as e:
                    if str(e) != 'The smallest edge difference is numerically 0.':
                        print("ValueError: ",e)
                    else:
                        nbins -= 1
            ## Capture cases of incomplete estimation
            if (nbins == 5) and not_finished:
                statistic = [np.nanmean(y_train[:,iv])]
                bin_centers = [np.nanmean(y_pred)]
                constant_input = True
            
            # Apply nonlinearity
            if np.sum(np.isnan(statistic)) > 0:  # For cases with NaN values (line would crash if no NaNs were found)
                y_pred_test_nl = np.interp(y_pred_test, bin_centers[~np.isnan(statistic)], statistic[~np.isnan(statistic)])
            else:
                y_pred_test_nl = np.interp(y_pred_test, bin_centers, statistic)
            
            # Scores
            r2_train = lr.score(X_train, y_train[:,iv])
            r2_test = lr.score(X_test, y_test[:,iv])
            pearsonr_score_linear = pearsonr(y_pred_test, y_test[:,iv]).statistic
            ## Correct for cases of failed nonlinearity estimation
            if constant_input:
                r2_nl = 0
                pearsonr_score = pearsonr_score_linear
            else:
                r2_nl = sklearn.metrics.r2_score(y_test[:,iv],y_pred_test_nl)
                pearsonr_score = pearsonr(y_pred_test_nl, y_test[:,iv]).statistic

            # Extract filter
            basis_weights = lr.coef_
            estimated_filters = basis_projection.inverse_transform(basis_weights)
            estimated_filters = estimated_filters.reshape((-1,window))

            # Store individual model results
            if not os.path.exists(f'{datefolder}'):
                os.mkdir(f'{datefolder}') 
            with open(f'{datefolder}/{filename}_{varname}.pkl', 'wb') as handle:
                pickle.dump({'model':lr,'pearsonr_score_linear':pearsonr_score_linear,'pearsonr_score':pearsonr_score,'r2_train':r2_train,'r2_test':r2_test,'r2_nl':r2_nl,'estimated_filters':estimated_filters,'y_pred_test':y_pred_test,'y_pred_test_nl':y_pred_test_nl,'bin_centers':bin_centers,'statistic':statistic,'y_means':y_means[iv],'y_stds':y_stds[iv],'nsamples':len(x), 'duration':len(x)/int(sample_frequency/bin_width), 'constant_input':constant_input, 'nbins':nbins,'y_train':y_train[:,iv],'y_test':y_test[:,iv],'y_pred_train':y_pred}, handle, protocol=pickle.HIGHEST_PROTOCOL)

  4%|▍         | 3/70 [00:51<19:25, 17.40s/it]C:\Users\apalaci\AppData\Local\Temp\ipykernel_13356\1568213949.py:66: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  pearsonr_score_linear = pearsonr(y_pred_test, y_test[:,iv]).statistic
 10%|█         | 7/70 [03:15<34:24, 32.77s/it]C:\Users\apalaci\AppData\Local\Temp\ipykernel_13356\1568213949.py:66: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  pearsonr_score_linear = pearsonr(y_pred_test, y_test[:,iv]).statistic
 13%|█▎        | 9/70 [03:37<21:29, 21.15s/it]C:\Users\apalaci\AppData\Local\Temp\ipykernel_13356\1568213949.py:66: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  pearsonr_score_linear = pearsonr(y_pred_test, y_test[:,iv]).statistic
C:\Users\apalaci\AppData\Local\Temp\ipykernel_13356\1568213949.py:66: NearConstantInputWarning: An input array is nearly constant; the computed correlatio

# Collect scores

In [5]:
# Will store each model performance and metadata in a single table

df_columns = ['filename','pearsonr_score_linear','pearsonr_score','r2_train','r2_test','r2_nl','varname','DN','DN_side','constant','nbins']
df_all = pd.DataFrame(columns=df_columns)
for index, row in df.iterrows():
    filename = row['filename']
    DN_side = row['side']
    DN = row['DN']
    for varname in y_names:
        res_file_path = f'{datefolder}/{filename}_{varname}.pkl'
        if os.path.exists(res_file_path):
            with open(res_file_path, 'rb') as handle:
                temp = pickle.load(handle)
                temp_df = pd.DataFrame([[filename, temp['pearsonr_score_linear'], temp['pearsonr_score'], temp['r2_train'], temp['r2_test'], temp['r2_nl'], varname, DN, DN_side, temp['constant_input'], temp['nbins']]], columns=df_columns)
            df_all = pd.concat((df_all,temp_df))
df_all.reset_index(drop=True,inplace=True)
df_all.to_pickle(f'{datefolder}/df_GLM_results.pkl')
df_all

C:\Users\apalaci\AppData\Local\Temp\ipykernel_13356\3880342856.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_all = pd.concat((df_all,temp_df))


,filename,pearsonr_score_linear,pearsonr_score,r2_train,r2_test,r2_nl,varname,DN,DN_side,constant,nbins
0,2022_06_08_0004,0.231351,0.246024,0.049869,0.048532,0.059141,v_fwd,imposter,right,False,32
1,2022_06_08_0004,0.283507,0.516329,0.026583,0.048715,0.250354,v_ang,imposter,right,False,32
2,2022_06_08_0004,0.502423,0.527277,0.299608,0.251683,0.277479,abs_v_fwd,imposter,right,False,32
3,2022_06_08_0004,0.605696,0.615813,0.399463,0.364030,0.360968,abs_v_ang,imposter,right,False,32
4,2022_06_08_0004,0.411937,0.435366,0.201621,0.166791,0.187986,pos_v_fwd,imposter,right,False,32
...,...,...,...,...,...,...,...,...,...,...,...
555,2024_11_29_0010,0.439826,0.451264,0.152477,-0.024915,0.067322,abs_v_ang,imposter,left,False,32
556,2024_11_29_0010,0.256785,0.279233,0.054347,-0.048387,-0.001645,pos_v_fwd,imposter,left,False,32
557,2024_11_29_0010,0.344122,0.329275,0.093031,0.031364,0.003998,pos_v_ang,imposter,left,False,32
558,2024_11_29_0010,0.282959,0.298908,0.049754,-0.022576,-0.000500,neg_v_fwd,imposter,left,False,32


# Collect filters

In [6]:
# Extract and store all filters per experiment and variable, and save as a dictionary

estimated_filters_dict = {}
for index, row in df.iterrows():
    filename = row['filename']
    DN_side = row['side']
    DN = row['DN']

    estimated_filters_dict[filename] = {}
    for varname in y_names:
        res_file_path = f'{datefolder}/{filename}_{varname}.pkl'
        if os.path.exists(res_file_path):
            estimated_filters_dict[filename][varname] = {}
            estimated_filters_list = []
            with open(res_file_path, 'rb') as handle:
                temp = pickle.load(handle)
                estimated_filters_list.append(temp['estimated_filters'][:])
            estimated_filters_dict[filename][varname] = np.vstack(estimated_filters_list)

with open(f'{datefolder}/GLM_filters.pkl', 'wb') as handle:
    pickle.dump(estimated_filters_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Collect nonlinearities

In [7]:
# Extract nonlinearity parameters for the correction of the linear predictions per experiment and variable, and save as a dictionary

nonlinearity_results = {}
for index, row in df.iterrows():
    filename = row['filename']
    DN_side = row['side']
    DN = row['DN']
    nonlinearity_results[filename] = {}
    for varname in y_names:
        nonlinearity_results[filename][varname] = {}
        res_file_path = f'{datefolder}/{filename}_{varname}.pkl'
        if os.path.exists(res_file_path):
            with open(res_file_path, 'rb') as handle:
                temp = pickle.load(handle)
                nonlinearity_results[filename][varname]['bin_centers'] = temp['bin_centers']
                nonlinearity_results[filename][varname]['statistic'] = temp['statistic']

with open(f'{datefolder}/nonlinearity_results.pkl', 'wb') as handle:
    pickle.dump(nonlinearity_results, handle, protocol=pickle.HIGHEST_PROTOCOL)